# Model Construction

## 1.Model

Cola class and Lola class

In [ ]:
import h5py
  
from keras.layers import *
from keras import Model
from keras.models import Sequential
from keras.layers.advanced_activations import PReLU
from keras import regularizers
import numpy as np
import keras, time
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt

class CoLa():
    """Combination layer that returns the input and appends some linear 
    combinations along the last dimension
    """
    def __init__(self, nAdded, **kwargs):
        # Set the number of added combinations
        self.nAdded = nAdded
        super(CoLa, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create trainable weight for the linear combination
        self.combination = self.add_weight(name='combination',
                    shape=(input_shape[2], self.nAdded),
                    initializer='uniform',
                    trainable=True)
        super(CoLa, self).build(input_shape)

    def call(self, x):
        # Generate combinations and return input with appended with combinations
        combined = K.dot(x, self.combination)
        return K.concatenate([x, combined], axis=2)

    def compute_output_shape(self, input_shape):
        self.out_shape = (input_shape[0], 
                    input_shape[1], 
                    input_shape[2] + self.nAdded)
        return self.out_shape

    def get_config(self):
        # Store nAdded value for loading saved models later
        base_config = super(CoLa, self).get_config()
        base_config['nAdded'] = self.nAdded
        return base_config


class LoLa():
    """Lorentz Layer adapted from arXiv:1707.08966
    From an input of 4 vectors generate some physical quantities that serve as
    input to a classifiction network
    """
    def __init__(self, **kwargs):
        super(LoLa, self).__init__(**kwargs)

    def build(self, input_shape):
        initializer = keras.initializers.TruncatedNormal(mean=0., stddev=0.1)
        metric = keras.initializers.Constant(value=[[1., -1., -1., -1.]])
        # Trainable metric for 4-vector multiplication
        self.metric = self.add_weight(name='metric',
                    shape=(1, 4),
                    initializer=metric,
                    trainable=True)

        # Weights for the linear combination of energies
        self.energyCombination = self.add_weight(name='energyCombination',
                    shape=(input_shape[-1], input_shape[-1]),
                    initializer=initializer,
                    trainable=True)
        
        # Weights for the linear combinations of distances
        self.distanceCombination = self.add_weight(name='distanceCombination',
                    shape=(input_shape[2], 4),
                    initializer=initializer,
                    trainable=True)
        super(LoLa, self).build(input_shape)

    def call(self, x):
        def getDistanceMatrix(x):
            """Input:
            x, (batchsize, features, nConst) - array of vectors
            Returns:
            dists, (batchsize, nConst, nConst) - distance array for every jet
            """
            part1 = -2 * K.batch_dot(x, K.permute_dimensions(x, (0, 2, 1)))
            part2 = K.permute_dimensions(K.expand_dims(K.sum(x**2, axis=2)), (0, 2, 1))
            part3 = K.expand_dims(K.sum(x**2, axis=2))
            dists = part1 + part2 + part3
            return dists

        # Get mass of each 4-momentum
        mass = K.dot(self.metric, K.square(x))
        mass = K.permute_dimensions(mass, (1, 0, 2))

        # Get pT of each 4-momentum
        pT = x[:, 1, :] ** 2 + x[:, 2, :] ** 2
        pT = K.sqrt(K.reshape(pT, (K.shape(pT)[0], 1, K.shape(pT)[1])))
        
        # Get a learnable linear combination of the energies of all constituents
        energies = K.dot(x[:, 0, :], self.energyCombination)
        energies = K.reshape(energies, 
                            (K.shape(energies)[0], 1, K.shape(energies)[1]))
 
        # Get the distance matrix and do some linear combination
        dists_3 = getDistanceMatrix(
                            K.permute_dimensions(x[:, 1:, :], (0, 2, 1)))
        dists_0 = getDistanceMatrix(
                            K.permute_dimensions(x[:, 0, None, :], (0, 2, 1)))
        dists = dists_0 - dists_3
        
        dists = K.dot(dists, self.distanceCombination)
        dists = K.permute_dimensions(dists, (0, 2, 1))

        return K.concatenate([mass, pT, energies, dists], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 7, input_shape[2])

In [ ]:
class LoLaClassifier(layers):
    def __init__(self, nConstituents, nAdded, tag=0):
        self.tag = tag
        self.model = self._genNetwork(nConstituents, nAdded)
        pass

    def _genNetwork(self, nConstituents, nAdded):
        input = Input((4, nConstituents))
        layer = input

        # Combination layer adds nAdded linear combinations of vectors
        layer = CoLa(nAdded=nAdded, name='cola')(layer)
        print (layer.value)
        # LoLa replaces the 4 vectors by physically more meaningful vectors
        layer = LoLa(name='lola')(layer) 
        print (layer)

        # Connect to a fully connected network for classification
        layer = Flatten()(layer)
        layer = Dense(100, activation='relu')(layer)
        #layer = Dense(50, activation='relu')(layer)
        layer = Dense(10, activation='relu')(layer)
        layer = Dense(2, activation='softmax')(layer)
        
        model = keras.Model(input, layer)
        return model

Now contruct the model.

In [ ]:
model = LoLaClassifier(nConstituents=40, nAdded=10)

In [ ]:
model.compile(
            optimizer=keras.optimizers.Adam(lr=0.0001), 
            loss='categorical_crossentropy', 
            metrics=['acc'])
print(model.summary())